# Overview

This notebook is used to create the hourly solar PV profiles that are used for the PyPSA-RSA model. The output file is a NetCDF file with the following dimensions:
- Technology type (Fixed Tilt, Tracking, Rooftop)
- Supply regions (bus)
- Intra region areas available for generation
- Hourly generation profiles (normalised)

Supply region resolution - 
- 1 - whole of RSA
- 10 - Eskom defined transmission regions
- 34 - Eskom defined local supply areas

Sub-region options:
- All area
- Renewable Energy Development Zones
- Power Corridors
- Active EIA applications
- REDZ + EIA applications 

Intra-region aggregation options:
- Average all available cells
- Weight cells by availability matrix
- Quantile

# Setting up ERA5 cutout using atlite

In [1]:
import atlite
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from dask.distributed import Client, LocalCluster
from IPython.display import clear_output
from _helpers import (
    load_gis_data,
    generate_pv_timeseries,
    aggregate_intra_region,
)

import warnings
warnings.filterwarnings('ignore')

In [2]:
agg_params = {
    "availability_threshold":0,
    "scale_by_availability": True,      
    "aggregation_method":"mean", # quantile
    "quantile": 0.2,
}
run_generation = True

# Load Atlite cutout
Can use ERA5 or Sarah data for solar PV

In [3]:
#Before running this script you must build an era5 cutout for South Africa using the atlite module
cutout = atlite.Cutout(
    path="../../cutouts/RSA-2017_22-sarah.nc",
    chunks={'time': 100}
)
cutout.data = cutout.data.sel(time=~((cutout.data.time.dt.month == 2) & (cutout.data.time.dt.day == 29)))


# Load GIS data

In [4]:
data_bundle_path = "../../data/bundle"
gis_data = load_gis_data(data_bundle_path)

Loading Eskom Supply Regions from ../../data/bundle/rsa_supply_regions.gpkg
Loading EIA applications from ../../data/bundle/REEA_OR_2023_Q3.shp
Loading REDZs from ../../data/bundle/REDZs.shp
Loading Power Corridors from ../../data/bundle/Power_corridors.shp
Loading SACAD from ../../data/bundle/SACAD_OR_2023_Q3.shp
Loading SAPAD from ../../data/bundle/SAPAD_OR_2023_Q3.shp
Loading SKA exclusion from ../../data/bundle/SKA/SKA_exclusion.shp


# Create Regional timeseries data for solar PV

In [6]:
for module in ["era5","sarah"]:
    print("Calculating timeseries data for Fixed Tilt, Single Axis Tracker and Rooftop PV")
    fixed_tilt_cf = generate_pv_timeseries(cutout, "Fixed Tilt", dc_ac_ratio = 1.15, module = module)
    sat_cf = generate_pv_timeseries(cutout, "Single Axis", dc_ac_ratio = 1.15, module = module)
    rooftop_cf = generate_pv_timeseries(cutout, "Rooftop", dc_ac_ratio = 1, module = module)

    for r in gis_data["supply_regions"].keys():
        availability_matrix = xr.open_dataarray(f"availability/availability_matrix_{r}.nc")
        rooftop_availability_matrix = xr.open_dataarray(f"availability/rooftop_availability_matrix_{r}.nc")
        timeseries = xr.DataArray(
            coords = {
                "time": cutout.data.coords["time"].values, 
                "bus": availability_matrix.coords["bus"], 
                "_type": ["Fixed Tilt", "Single Axis", "Rooftop"], 
                "intra_region": ["all", "redz", "corridors", "redz_corridors_eia"]},
            dims = ["time", "bus", "_type", "intra_region"],
        )

        for intra_region in timeseries.coords["intra_region"].values:
            for bus in availability_matrix.coords["bus"].values:
                print(f"Calculating average based on intra region area: {intra_region} for bus: {bus}")
                region_availability = availability_matrix.sel(area=intra_region,bus=bus).rename({"y":"lat","x":"lon"})
                rooftop_region_availability = rooftop_availability_matrix.sel(bus=bus).rename({"y":"lat","x":"lon"})
                timeseries.loc[dict(_type="Fixed Tilt", bus=bus, intra_region=intra_region)] = aggregate_intra_region(fixed_tilt_cf, region_availability, **agg_params)
                timeseries.loc[dict(_type="Single Axis", bus=bus, intra_region=intra_region)] = aggregate_intra_region(sat_cf, region_availability, **agg_params)
                timeseries.loc[dict(_type="Rooftop", bus=bus, intra_region=intra_region)] = aggregate_intra_region(rooftop_cf, rooftop_region_availability, **agg_params)
                clear_output(wait=True)  

        timeseries.to_netcdf(f"timeseries_data/extendable_solar_pv_{module}_{r}.nc")

Calculating average based on intra region area: redz_corridors_eia for bus: Witkop


In [ ]:
for module in ["csir"]:

    for r in [1, 27]:
        availability_matrix = xr.open_dataarray(f"availability/availability_matrix_{r}.nc")
        csir_fise = pd.read_excel("csir_fise_SWA_data.xlsx", sheet_name = f"{r}-solar_pv", index_col=0, parse_dates=True)
        csir_fise = csir_fise.iloc[1:]
        csir_fise.index = pd.DatetimeIndex(csir_fise.index)
        csir_fise = csir_fise[~((csir_fise.index.month == 2) & (csir_fise.index.day == 29))]

        csir_fise = pd.concat(
            [csir_fise]
            * (len(cutout.data.coords["time"].values) // len(csir_fise)),
            ignore_index=True,
        )  

        timeseries = xr.DataArray(
            coords = {
                "time": cutout.data.coords["time"].values, 
                "bus": availability_matrix.coords["bus"], 
                "_type": ["Fixed Tilt", "Single Axis", "Rooftop"], 
                "intra_region": ["all", "redz", "corridors", "redz_corridors_eia"]},
            dims = ["time", "bus", "_type", "intra_region"],
        )

        for intra_region in timeseries.coords["intra_region"].values:
            for bus in availability_matrix.coords["bus"].values:
                print(f"Calculating average based on intra region area: {intra_region} for bus: {bus}")
                timeseries.loc[dict(_type="Fixed Tilt", bus=bus, intra_region=intra_region)] = csir_fise[bus].values

        timeseries.to_netcdf(f"timeseries_data/extendable_solar_pv_{module}_{r}.nc")

Calculating average based on intra region area: all for bus: RSA
Calculating average based on intra region area: redz for bus: RSA
Calculating average based on intra region area: corridors for bus: RSA
Calculating average based on intra region area: redz_corridors_eia for bus: RSA
Calculating average based on intra region area: all for bus: Namaqualand
Calculating average based on intra region area: all for bus: West Coast
Calculating average based on intra region area: all for bus: Peninsula
Calculating average based on intra region area: all for bus: Southern Cape
Calculating average based on intra region area: all for bus: Port Elizabeth
Calculating average based on intra region area: all for bus: Karoo
Calculating average based on intra region area: all for bus: Kimberley
Calculating average based on intra region area: all for bus: Carletonville
Calculating average based on intra region area: all for bus: Welkom
Calculating average based on intra region area: all for bus: Bloemfont